## Load latest headlines

In [1]:
# ========== Bootstrap: ensure required Python packages are present =========
import importlib, subprocess, sys
from typing import Optional

def _ensure(pkg_name: str, import_name: Optional[str] = None):
    try:
        importlib.import_module(import_name or pkg_name)
    except ModuleNotFoundError:
        subprocess.check_call([sys.executable, '-m', 'pip', 'install', pkg_name])
    finally:
        globals()[import_name or pkg_name] = importlib.import_module(import_name or pkg_name)

_ensure('pandas')
print('All dependencies ready.\n')


All dependencies ready.



In [2]:
from pathlib import Path
archive = Path('archive')
archive.mkdir(exist_ok=True)
import pandas as pd
latest = pd.read_csv('../headlines/latest.csv')
latest.head()

,pubdate,source,title,link
0,2025-09-15-20-50-24 +0000,cbc,Trump says U.S. has struck another alleged dru...,https://www.cbc.ca/news/world/us-drug-boat-ven...
1,2025-09-15-20-49-00 +0000,wsj,Opinion | A Week That Shook the Old Order,https://www.wsj.com/opinion/a-week-that-shook-...
2,2025-09-15-20-48-11 +0000,nypost,"Long Island lawmaker, Marine will walk 26 hour...",https://nypost.com/2025/09/15/us-news/long-isl...
3,2025-09-15-20-48-00 +0000,wsj,U.S. Strikes Second Alleged Drug Boat From Ven...,https://www.wsj.com/world/americas/u-s-strikes...
4,2025-09-15-20-48-00 +0000,wsj,Opinion | Pray for Tyler Robinson’s Family,https://www.wsj.com/opinion/pray-for-tyler-rob...


## Calculate word frequencies

In [3]:
import re
from collections import Counter
from datetime import datetime

with open('exclude.txt') as f:
    stop_words = set(w.strip() for w in f if w.strip())
words = re.findall(r'[A-Za-z]+', ' '.join(latest['title']).lower())
filtered = [w for w in words if w not in stop_words and len(w) > 1]
counts = Counter(filtered)
score_df = (
    pd.DataFrame(counts.items(), columns=['word','score'])
    .sort_values('score', ascending=False)
)
score_df[['score','word']].to_csv('scores.csv', index=False)
timestamp = datetime.utcnow().strftime('%Y-%m-%d-%H-00-00')
score_df[['score','word']].to_csv(archive / f'scores-{timestamp}.csv', index=False)
score_df.head()


/tmp/ipykernel_2314/2220774760.py:15: DeprecationWarning: datetime.datetime.utcnow() is deprecated and scheduled for removal in a future version. Use timezone-aware objects to represent datetimes in UTC: datetime.datetime.now(datetime.UTC).
  timestamp = datetime.utcnow().strftime('%Y-%m-%d-%H-00-00')


,word,score
0,trump,44
30,kirk,31
29,charlie,27
57,new,14
51,suspect,13


## Rank headlines by score

In [4]:
word_scores = dict(score_df[['word','score']].values)
latest['score'] = latest['title'].apply(
    lambda t: sum(
        word_scores.get(w.lower(), 0)
        for w in re.findall(r'[A-Za-z]+', t)
        if len(w) > 1
    )
)
ranked = latest.sort_values('score', ascending=False)
ranked[['score','pubdate','source','title','link']].to_csv('rank.csv', index=False)
ranked[['score','pubdate','source','title','link']].to_csv(archive / f'rank-{timestamp}.csv', index=False)
ranked.head()


,pubdate,source,title,link,score
266,2025-09-14-23-51-33 +0000,nypost,Trump blasts ‘the left’ and announces ‘major i...,https://nypost.com/2025/09/14/us-news/trump-bl...,123
40,2025-09-15-19-30-07 +0000,nypost,Charlie Kirk shooting suspect Tyler Robinson a...,https://nypost.com/2025/09/15/us-news/charlie-...,106
110,2025-09-15-16-57-00 +0000,nypost,"On ‘Charlie Kirk Show’ hosted by JD Vance, Ste...",https://nypost.com/2025/09/15/us-news/jd-vance...,100
56,2025-09-15-18-54-19 +0000,latimes,Suspect sent text saying he would 'take out' C...,https://www.latimes.com/california/story/2025-...,100
68,2025-09-15-18-27-22 +0000,wapo,‘It was me’: Suspect in Charlie Kirk shooting ...,https://www.washingtonpost.com/investigations/...,93


## Select top headlines

In [5]:
top_rows = []
working = word_scores.copy()
remaining = latest.copy()
for _ in range(10):
    ranked_loop = remaining.assign(score=remaining['title'].apply(
        lambda t: sum(working.get(w.lower(), 0)
                      for w in re.findall(r'[A-Za-z]+', t)
                      if len(w) > 1)
    )).sort_values('score', ascending=False)
    if ranked_loop.empty:
        break
    top_story = ranked_loop.iloc[0]
    top_rows.append(top_story[['score','pubdate','source','title','link']])
    words = set(re.findall(r'[A-Za-z]+', top_story['title'].lower()))
    for w in words:
        working.pop(w, None)
    remaining = remaining.drop(top_story.name)
top_df = pd.DataFrame(top_rows)
top_df.to_csv('top.csv', index=False)
top_df.to_csv(archive / f'top-{timestamp}.csv', index=False)
top_df


,score,pubdate,source,title,link
266,123,2025-09-14-23-51-33 +0000,nypost,Trump blasts ‘the left’ and announces ‘major i...,https://nypost.com/2025/09/14/us-news/trump-bl...
160,45,2025-09-15-14-03-11 +0000,nypost,"Bessent says US, China have reached ‘framework...",https://nypost.com/2025/09/15/business/us-chin...
134,45,2025-09-15-15-15-00 +0000,wsj,Canadian factories notched the biggest rise in...,https://www.wsj.com/articles/canada-factory-an...
40,43,2025-09-15-19-30-07 +0000,nypost,Charlie Kirk shooting suspect Tyler Robinson a...,https://nypost.com/2025/09/15/us-news/charlie-...
13,43,2025-09-15-20-29-55 +0000,cbc,"Carney, Poilievre come face-to-face for the fi...",https://www.cbc.ca/news/politics/carney-poilie...
125,38,2025-09-15-15-48-34 +0000,nyt,Elon Musk Buys $1 Billion in Tesla Stock as Bo...,https://www.nytimes.com/2025/09/15/business/el...
110,36,2025-09-15-16-57-00 +0000,nypost,"On ‘Charlie Kirk Show’ hosted by JD Vance, Ste...",https://nypost.com/2025/09/15/us-news/jd-vance...
2,35,2025-09-15-20-48-11 +0000,nypost,"Long Island lawmaker, Marine will walk 26 hour...",https://nypost.com/2025/09/15/us-news/long-isl...
137,34,2025-09-15-15-04-49 +0000,nypost,Top GOP foreign-policy Rep. Michael McCaul won...,https://nypost.com/2025/09/15/us-news/texas-re...
3,32,2025-09-15-20-48-00 +0000,wsj,U.S. Strikes Second Alleged Drug Boat From Ven...,https://www.wsj.com/world/americas/u-s-strikes...


## Create JSON version of top headlines

In [6]:
import pandas as pd
pd.read_csv('top.csv').to_json('top.json', orient='records', indent=2)
